# Image cropping function in action
## load data

In [2]:
import scanpy as sc
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import time

from spatial_tools.image._utils import *
from spatial_tools.image.manipulate import crop_img
from spatial_tools.image.tools import read_tif, get_image_features

import skimage.feature as sk_image

# path to "raw" dataset folder
BASE_PATH = "/storage/groups/ml01/datasets/raw/20200909_PublicVisium_giovanni.palla"
dataset_name = "V1_Adult_Mouse_Brain"
dataset_folder = os.path.join(
    BASE_PATH, "20191205_10XVisium_MouseBrainCoronal_giovanni.palla"
)

## example of calculating the feature table, currently just supporting hog features

In [6]:
adata = sc.read_visium(
    dataset_folder, count_file=f"{dataset_name}_filtered_feature_bc_matrix.h5"
)

# select which features to add
features = features=["hog", "texture", "summary", "color_hist"]
feature_table = get_image_features(adata[0:1], dataset_folder, dataset_name)
feature_table

Variable names are not unique. To make them unique, call `.var_names_make_unique`.
/home/icb/olle.holmberg/anaconda3/envs/hackathon-spatial/lib/python3.6/site-packages/anndata/_core/anndata.py:1094: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  if not is_categorical(df_full[k]):
Variable names are not unique. To make them unique, call `.var_names_make_unique`.


,summary_quantile_0.9_ch_0,summary_quantile_0.5_ch_0,summary_quantile_0.1_ch_0,summary_quantile_0.9_ch_1,summary_quantile_0.5_ch_1,summary_quantile_0.1_ch_1,summary_quantile_0.9_ch_2,summary_quantile_0.5_ch_2,summary_quantile_0.1_ch_2
cell_name,,,,,,,,,
AAACAAGTATCTCCCA-1,132.0,111.0,77.0,102.0,79.0,54.0,138.0,111.0,88.0
AAACAATCTACTAGCA-1,140.0,111.0,80.0,87.0,60.0,39.0,135.0,108.0,84.0
AAACACCAATAACTGC-1,132.0,116.0,89.0,109.0,91.0,67.0,130.0,117.0,97.0
AAACAGAGCGACTCCT-1,136.0,116.0,93.0,115.0,82.0,58.0,137.0,113.0,89.0
AAACCGGGTAGGTACC-1,137.0,112.0,83.0,103.0,78.0,54.0,133.0,113.0,92.0
AAACCGTTCGTCCAGG-1,141.0,104.0,59.0,105.0,70.0,39.0,145.0,112.0,80.0
AAACCTCATGAAGTTG-1,133.0,111.0,74.0,100.0,79.0,55.0,132.0,114.0,92.0
AAACGAAGAACATACC-1,136.0,109.0,74.0,86.0,60.0,37.0,137.0,108.0,84.0
AAACGAGACGGTTGAT-1,134.0,106.0,65.0,94.0,67.0,37.0,146.0,112.0,75.0


In [ ]:
adata = sc.read_visium(
    dataset_folder, count_file=f"{dataset_name}_filtered_feature_bc_matrix.h5"
)
img = read_tif(dataset_folder, dataset_name)

xcoord = adata.obsm["spatial"][:, 0]
ycoord = adata.obsm["spatial"][:, 1]
spot_diameter = adata.uns['spatial'][dataset_name]['scalefactors']['spot_diameter_fullres']
spot_id = 1
crop_1 = crop_img(img, xcoord[spot_id], ycoord[spot_id], scalef=1, sizef=1, spot_diameter=spot_diameter)
cell_name = adata.obs.index[0]
get_summary_stats(crop_1, "summary")

## get feature table with multi threading

In [ ]:
from tqdm import tqdm
from joblib import Parallel, delayed

def process(andata_feature, spot_id, cell_name):
    features = ["hog"]
    crop_ = crop_img(andata_feature.image, andata_feature.xcoord[spot_id], andata_feature.ycoord[spot_id], scalef=1, 
                          sizef=1, spot_diameter=andata_feature.spot_diameter)
        
    features_pd = get_features_statistics(crop_, cell_name, features=andata_feature.features)
    return features_pd

@timing
def get_features_abt_multithread(cell_names, num_cores = 1):
    processed_list = Parallel(n_jobs = num_cores)(delayed(process)(andata_feature, iter_, i) for iter_, 
                                              i in enumerate(cell_names))
    
    features_abt = pd.concat(processed_list)
    features_abt.set_index(["cell_name"], inplace=True)
    return features_abt
    
    
class AndataFeature():
    def __init__(self, andata, dataset_folder, dataset_name, features=["hog"]):
        self.image = read_tif(dataset_folder, dataset_name)
        self.xcoord = andata.obsm["spatial"][:, 0]
        self.ycoord = andata.obsm["spatial"][:, 1]
        self.spot_diameter = andata.uns['spatial'][dataset_name]['scalefactors']['spot_diameter_fullres']
        self.features = features

In [ ]:
## example on how to retrieve 

In [ ]:
from tqdm import tqdm
from joblib import Parallel, delayed
    
class AndataFeature():
    def __init__(self, andata, dataset_folder, dataset_name, features=["hog"]):
        self.image = read_tif(dataset_folder, dataset_name)
        self.xcoord = andata.obsm["spatial"][:, 0]
        self.ycoord = andata.obsm["spatial"][:, 1]
        self.spot_diameter = andata.uns['spatial'][dataset_name]['scalefactors']['spot_diameter_fullres']
        self.features = features
        
    def process(self, cell_name):
        features = ["hog"]
        spot_id = adata.obs.index.get_loc(cell_name)
        crop_ = crop_img(self.image, self.xcoord[spot_id], self.ycoord[spot_id], scalef=1, 
                              sizef=1, spot_diameter=self.spot_diameter)

        features_pd = get_features_statistics_df(crop_, cell_name, features=self.features)
        return features_pd

    @timing
    def get_features_abt_multithread(self, cell_names, num_cores = 1):
        processed_list = Parallel(n_jobs = num_cores, backend="threading")(delayed(self.process)(i) for i in cell_names)

        features_abt = pd.concat(processed_list)
        features_abt.set_index(["cell_name"], inplace=True)
        return features_abt

In [ ]:
if __name__ == '__main__':
    adata = sc.read_visium(
        dataset_folder, count_file=f"{dataset_name}_filtered_feature_bc_matrix.h5"
    )

    andata_feature = AndataFeature(andata = adata, dataset_folder=dataset_folder, 
                               dataset_name=dataset_name, features=["hog"])

    cell_names = adata[0:2000].obs.index.tolist()
    andata_feature.get_features_abt_multithread(cell_names , num_cores=5)

In [ ]:
# get cell names
cell_names = adata[0:10].obs.index.tolist()
get_features_abt_multithread(cell_names, num_cores = 1)

## crop image
- use different sizefactors and scalefactors
- try masking

location of the spot that we are cropping

In [ ]:
spot_id = 100
plt.scatter(xcoord[spot_id], ycoord[spot_id], c='green')
plt.imshow(img)

crop with different neighborhood sizes. Note that the function also works when the range is outside the image

In [ ]:
crop_1 = crop_img(img, xcoord[spot_id], ycoord[spot_id], scalef=1, sizef=1, spot_diameter=spot_diameter)
crop_2 = crop_img(img, xcoord[spot_id], ycoord[spot_id], scalef=1, sizef=2, spot_diameter=spot_diameter)
crop_10 = crop_img(img, xcoord[spot_id], ycoord[spot_id], scalef=1, sizef=10, spot_diameter=spot_diameter)
crop_100 = crop_img(img, xcoord[spot_id], ycoord[spot_id], scalef=1, sizef=100, spot_diameter=spot_diameter)

fig, axes = plt.subplots(2,2)
axes[0][0].imshow(crop_1)
axes[0][1].imshow(crop_2)
axes[1][0].imshow(crop_10)
axes[1][1].imshow(crop_100)

In [ ]:
dict_ = {"1_hog":1,"2_hog":2, "stat_4":4,"stat_5": 5}
log = pd.DataFrame([dict_])
log["cell_name"] = "test"
log

crop with different scales - note how the crops get smaller with smaller `scalef`

In [ ]:
crop_1 = crop_img(img, xcoord[spot_id], ycoord[spot_id], scalef=1, sizef=10, spot_diameter=spot_diameter)
crop_05 = crop_img(img, xcoord[spot_id], ycoord[spot_id], scalef=0.5, sizef=10, spot_diameter=spot_diameter)
crop_025 = crop_img(img, xcoord[spot_id], ycoord[spot_id], scalef=0.25, sizef=10, spot_diameter=spot_diameter)
crop_01 = crop_img(img, xcoord[spot_id], ycoord[spot_id], scalef=0.1, sizef=10, spot_diameter=spot_diameter)

fig, axes = plt.subplots(2,2)
axes[0][0].imshow(crop_1)
axes[0][1].imshow(crop_05)
axes[1][0].imshow(crop_025)
axes[1][1].imshow(crop_01)